<a href="https://colab.research.google.com/github/Paulina9555/kursGIT/blob/master/train_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq keras_bert
!pip install -qq tokenization
!pip install -qq bert-tensorflow
!pip install -qq transformers

!pip install -qq catboost
!pip install -qq xgboost
!pip install -qq lightgbm

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
from keras_bert import load_trained_model_from_checkpoint
import pandas as pd
from transformers import BertTokenizer

import catboost as ctb
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [ ]:
def init_tokenizer_and_load_bert(model_name='uncased_L-2_H-128_A-2', do_lower_case=True, model_trainable=False):
    model_dir = 'models/{}'.format(model_name)

    config_path = model_dir + '/bert_config.json'
    checkpoint_path = model_dir +'/bert_model.ckpt'
    vocab_path = model_dir + '/vocab.txt'

    print("loading: {}".format(model_name))

    tokenizer = BertTokenizer(vocab_path)
    print("vocab size: {}".format(len(tokenizer.vocab)))

    model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=model_trainable)
    print("loaded: {}".format(model_name))

    return tokenizer, model

In [ ]:
cd models

/content/models


In [ ]:
ls

uncased_L-2_H-128_A-2.zip


In [ ]:
!unzip "uncased_L-2_H-128_A-2.zip"

Archive:  uncased_L-2_H-128_A-2.zip
  inflating: uncased_L-2_H-128_A-2/bert_config.json  
  inflating: uncased_L-2_H-128_A-2/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-2_H-128_A-2/bert_model.ckpt.index  
  inflating: uncased_L-2_H-128_A-2/vocab.txt  


In [ ]:
cd ..

/content


In [ ]:
tokenizer, bert_model = init_tokenizer_and_load_bert("uncased_L-2_H-128_A-2")

loading: uncased_L-2_H-128_A-2
vocab size: 30522


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


loaded: uncased_L-2_H-128_A-2


In [ ]:
sent= 'how to train the model'

In [ ]:
res = tokenizer.encode_plus(sent, max_length=32, padding='max_length')
res

{'input_ids': [101, 2129, 2000, 3345, 1996, 2944, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
tokenizer.decode(res['input_ids'])

'[CLS] how to train the model [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
res = tokenizer.encode_plus(sent, max_length=512, padding='max_length', truncation=True)
len(res['input_ids'])

512

In [ ]:
tokenizer.decode(res['input_ids'])

'[CLS] how to train the model [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [ ]:
df = pd.read_hdf('tweets_airline.h5')
df = df.sample(5000)
print(df.shape)

df.sample(5)

(5000, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
8616,568107822275305472,positive,0.6728,unknown,NaN,Delta,NaN,History_Dork,NaN,0,@JetBlue The Magic eight ball has never steere...,NaN,2015-02-18 10:00:34 -0800,right behind you,Eastern Time (US & Canada)
7634,569489114707431424,negative,0.6575,Can't Tell,0.3394,Delta,NaN,betorides,NaN,0,@JetBlue I travel for business twice a week an...,NaN,2015-02-22 05:29:20 -0800,NaN,Quito
1621,569641805073031168,negative,0.6966,Customer Service Issue,0.3596,United,NaN,DanVidayo,NaN,0,@united yes was trying to upgrade my seat. Now...,NaN,2015-02-22 15:36:04 -0800,Earth,Indiana (East)
14575,569596793941401601,negative,1.0000,Customer Service Issue,0.6521,American,NaN,rdaniel10,NaN,0,@AmericanAir don't you guys have an email addr...,NaN,2015-02-22 12:37:12 -0800,NaN,Eastern Time (US & Canada)
5270,569223862941929472,positive,0.6778,unknown,NaN,Southwest,NaN,SouthwestAir,NaN,7,@SouthwestAir y'all are fast we already have o...,NaN,2015-02-21 11:55:19 -0800,"Dallas, Texas",Central Time (US & Canada)


In [ ]:
df['airline_sentiment'].isnull().sum()

0

In [ ]:
df['text'] = df['text'].str.lower()

In [ ]:
tokenize = lambda sent: tokenizer.encode_plus(sent, max_length=512, padding='max_length', truncation=True)
%time df['tokens'] = df['text'].map(tokenize)

CPU times: user 2.55 s, sys: 34.6 ms, total: 2.58 s
Wall time: 2.58 s


In [ ]:
df['tokens']

9011     [input_ids, token_type_ids, attention_mask]
12771    [input_ids, token_type_ids, attention_mask]
9370     [input_ids, token_type_ids, attention_mask]
9577     [input_ids, token_type_ids, attention_mask]
7604     [input_ids, token_type_ids, attention_mask]
                            ...                     
6056     [input_ids, token_type_ids, attention_mask]
2240     [input_ids, token_type_ids, attention_mask]
7701     [input_ids, token_type_ids, attention_mask]
6426     [input_ids, token_type_ids, attention_mask]
1841     [input_ids, token_type_ids, attention_mask]
Name: tokens, Length: 5000, dtype: object

In [ ]:
df['input_ids'] = df['tokens'].map(lambda t: t['input_ids'] )
df['token_type_ids'] = df['tokens'].map(lambda t: t['token_type_ids'] )
df['attention_mask'] = df['tokens'].map(lambda t: t['attention_mask'] )

In [ ]:
df.head(2)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,tokens,input_ids,token_type_ids,attention_mask
9011,570287318264905729,neutral,0.6762,unknown,NaN,US Airways,NaN,gleenc,NaN,0,@usairways any travel advisory coming for the ...,NaN,2015-02-24 10:21:06 -0800,NaN,NaN,"[input_ids, token_type_ids, attention_mask]","[101, 1030, 3915, 4313, 14035, 2151, 3604, 731...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
12771,570031615130394624,positive,0.6708,unknown,NaN,American,NaN,Princessk91,NaN,0,@americanair thanks 😩. idk if it still late fl...,NaN,2015-02-23 17:25:02 -0800,☀️SoFla☀️,Atlantic Time (Canada),"[input_ids, token_type_ids, attention_mask]","[101, 1030, 25988, 4313, 4283, 100, 1012, 8909...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
input_ids = np.stack(df['input_ids'])
token_type_ids = np.stack(df['token_type_ids'])
attention_mask = np.stack(df['attention_mask'])


input_ids.shape, token_type_ids.shape, attention_mask.shape

((5000, 512), (5000, 512), (5000, 512))

In [ ]:
input_ids

array([[  101,  1030,  3915, ...,     0,     0,     0],
       [  101,  1030, 25988, ...,     0,     0,     0],
       [  101,  1030,  3915, ...,     0,     0,     0],
       ...,
       [  101,  1030,  6892, ...,     0,     0,     0],
       [  101,  1030,  4943, ...,     0,     0,     0],
       [  101,  1030,  2142, ...,     0,     0,     0]])

In [ ]:
tokenizer, bert_model = init_tokenizer_and_load_bert("uncased_L-2_H-128_A-2")

loading: uncased_L-2_H-128_A-2
vocab size: 30522


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


loaded: uncased_L-2_H-128_A-2


In [ ]:
%time predicts = bert_model.predict([input_ids, token_type_ids], verbose=1)

157/157 [==============================] - 4s 24ms/step
CPU times: user 1.63 s, sys: 1.11 s, total: 2.74 s
Wall time: 5.45 s


In [ ]:
predicts.shape

(5000, 512, 128)

In [ ]:
X = predicts[:, 0 , :]
X.shape

(5000, 128)

In [ ]:
X = np.mean(predicts, axis = 1)

In [ ]:
y = df['airline_sentiment'].map({"negative":0,
                                 "positive": 1,
                                 "neutral":2})

In [ ]:
model = lgb.LGBMClassifier(max_depth=7, n_estimators=500, random_state=0, verbose = -1)
%time scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')

np.mean(scores), np.std(scores)

CPU times: user 52.9 s, sys: 225 ms, total: 53.1 s
Wall time: 53.1 s


(0.7324015388999031, 0.006303856487358055)

In [ ]:
model = xgb.XGBClassifier(max_depth=7, n_estimators=500, random_state=0, verbose = -1)
%time scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')

np.mean(scores), np.std(scores)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:31:06] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:31:22] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:31:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


CPU times: user 48.2 s, sys: 256 ms, total: 48.4 s
Wall time: 48.8 s


(0.7274008583637414, 0.0038996736564818686)

In [ ]:
model = ctb.CatBoostClassifier(max_depth=7, n_estimators=500, random_state=0)
%time scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')

np.mean(scores), np.std(scores)

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = tree.DecisionTreeClassifier(max_depth = 7)
%time scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')

np.mean(scores), np.std(scores)

CPU times: user 943 ms, sys: 3.02 ms, total: 946 ms
Wall time: 944 ms


(0.6437986112261341, 0.005125211887410224)

In [ ]:
model = RandomForestClassifier(max_depth = 7, n_estimators=500)
%time scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')

np.mean(scores), np.std(scores)

CPU times: user 30.8 s, sys: 44.6 ms, total: 30.9 s
Wall time: 31 s


(0.6904008153951442, 0.003354055734293983)

In [ ]:
from sklearn.svm import SVC
model = SVC(gamma='auto')
%time scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')

np.mean(scores), np.std(scores)

CPU times: user 3.24 s, sys: 12 ms, total: 3.25 s
Wall time: 3.24 s


(0.7292008585077703, 0.004579109335366435)